In [13]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa


In [14]:
TRAINING_DIR = 'F:/Xray_Gaus/train'
TEST_DIR='F:/Xray_Gaus/test'
VAL_DIR='F:/Xray_Gaus/val'


In [15]:
num_classes = 2
IMG_SIZE = 224
BATCH_SIZE = 32
input_shape = (224,224, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 10

In [16]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['COVID-19','pneumonia'],
    subset='training'
) 

Found 5011 images belonging to 2 classes.


In [17]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
val_gen = val_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['COVID-19','pneumonia'],
    subset='training'
)

Found 625 images belonging to 2 classes.


In [18]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = test_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        class_mode='binary',
    classes=['COVID-19','pneumonia'],
        #subset='validation'
)

Found 624 images belonging to 2 classes.


In [21]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

from tensorflow.keras.optimizers import Adam

base_model = keras.applications.DenseNet169(
    weights = 'imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False)

base_model.trainable = False

x = base_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation='softmax')(x)

# se acopla el modelo
model = keras.Model(base_model.input, outputs)

# congelar capas
for layer in base_model.layers:
    layer.trainable = False

# compilar el modelo.
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model.summary()

___________
conv5_block21_0_relu (Activatio (None, 7, 7, 1280)   0           conv5_block21_0_bn[0][0]         
__________________________________________________________________________________________________
conv5_block21_1_conv (Conv2D)   (None, 7, 7, 128)    163840      conv5_block21_0_relu[0][0]       
__________________________________________________________________________________________________
conv5_block21_1_bn (BatchNormal (None, 7, 7, 128)    512         conv5_block21_1_conv[0][0]       
__________________________________________________________________________________________________
conv5_block21_1_relu (Activatio (None, 7, 7, 128)    0           conv5_block21_1_bn[0][0]         
__________________________________________________________________________________________________
conv5_block21_2_conv (Conv2D)   (None, 7, 7, 32)     36864       conv5_block21_1_relu[0][0]       
_________________________________________________________________________________________________

In [22]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = 5)

Epoch 1/5
156/156 [==============================] - 575s 4s/step - loss: 0.8423 - accuracy: 0.9619 - precision: 0.9619 - recall: 1.0000 - val_loss: 0.7431 - val_accuracy: 0.9671 - val_precision: 0.9671 - val_recall: 1.0000
Epoch 2/5
156/156 [==============================] - 518s 3s/step - loss: 0.1196 - accuracy: 0.9690 - precision: 0.9690 - recall: 1.0000 - val_loss: 0.1280 - val_accuracy: 0.9671 - val_precision: 0.9671 - val_recall: 1.0000
Epoch 3/5
156/156 [==============================] - 528s 3s/step - loss: 0.1211 - accuracy: 0.9627 - precision: 0.9627 - recall: 1.0000 - val_loss: 0.0983 - val_accuracy: 0.9704 - val_precision: 0.9704 - val_recall: 1.0000
Epoch 4/5
156/156 [==============================] - 524s 3s/step - loss: 0.0842 - accuracy: 0.9640 - precision: 0.9640 - recall: 1.0000 - val_loss: 0.0710 - val_accuracy: 0.9671 - val_precision: 0.9671 - val_recall: 1.0000
Epoch 5/5
156/156 [==============================] - 508s 3s/step - loss: 0.0704 - accuracy: 0.9674 - pr

In [23]:
model.save('F:/Xray_Gaus/Densenet169_GausAd_binary_wt')

history.history

INFO:tensorflow:Assets written to: F:/Xray_Gaus/Densenet169_GausAd_binary_wt\assets


{'loss': [0.4179612100124359,
  0.11593794077634811,
  0.09962248802185059,
  0.08377324789762497,
  0.07852745801210403],
 'accuracy': [0.9650532007217407,
  0.9650532007217407,
  0.9654549360275269,
  0.9652540683746338,
  0.9650532007217407],
 'precision': [0.9650532007217407,
  0.9650532007217407,
  0.9654549360275269,
  0.9652540683746338,
  0.9650532007217407],
 'recall': [1.0, 1.0, 1.0, 1.0, 1.0],
 'val_loss': [0.74309903383255,
  0.1279645413160324,
  0.09832552075386047,
  0.07100959122180939,
  0.10859733074903488],
 'val_accuracy': [0.9671052694320679,
  0.9671052694320679,
  0.9703947305679321,
  0.9671052694320679,
  0.96875],
 'val_precision': [0.9671052694320679,
  0.9671052694320679,
  0.9703947305679321,
  0.9671052694320679,
  0.96875],
 'val_recall': [1.0, 1.0, 1.0, 1.0, 1.0]}

In [24]:
results = model.evaluate(test_gen, batch_size=128)

20/20 [==============================] - 46s 2s/step - loss: 0.0666 - accuracy: 0.9679 - precision: 0.9679 - recall: 1.0000


In [25]:
y_pred = model.predict(test_gen, verbose=1)
y_pred = np.argmax(y_pred, axis = 1)

20/20 [==============================] - 47s 2s/step


In [26]:
y_true=test_gen.labels

In [27]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.03      1.00      0.06        20
           1       0.00      0.00      0.00       604

    accuracy                           0.03       624
   macro avg       0.02      0.50      0.03       624
weighted avg       0.00      0.03      0.00       624

C:\Users\mprad\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[ 20,   0],
       [604,   0]], dtype=int64)